In [219]:
import math
import random
class NeuralNetwork:
    Learning_Rate = 0.05
    def __init__(self,num_inputs,num_hidden,num_outputs,hidden_layer_weights=None,output_layer_weights=None,hidden_layer_bias=None,output_layer_bias=None):
        self.num_inputs = num_inputs 
        self.hidden_layer = NeuronLayer(num_hidden, hidden_layer_bias)
        self.output_layer = NeuronLayer(num_outputs, output_layer_bias)
        
        self.init_weight1(hidden_layer_weights)
        self.init_weight2(output_layer_weights)
        
    #Weight from input to hidden layer
    def init_weight1(self,hidden_layer_weight):
        weight_num = 0
        for h in range(len(self.hidden_layer.neurons)):
            for i in range(self.num_inputs):
                self.hidden_layer.neurons[h].weights.append(hidden_layer_weight[weight_num])
                weight_num += 1
    
    #Weight from hidden to output
    def init_weight2(self,output_layer_weight):
        weight_num = 0
        for o in range(len(self.output_layer.neurons)):
            for h in range(len(self.hidden_layer.neurons)):
                self.output_layer.neurons[o].weights.append(output_layer_weight[weight_num])
                weight_num += 1
    
    def feed_forward(self,inputs):
        hidden_layer_outputs = self.hidden_layer.feed_forward(inputs)
        return self.output_layer.feed_forward(hidden_layer_outputs)
    
    def train(self,train_inputs,train_outputs):
        self.feed_forward(train_inputs)
        
        pd_error_total_output_net = [0]*len(self.output_layer.neurons)
        for o in range(len(self.output_layer.neurons)):
            #calculate the ∂E(total_output_net)/∂E(net_output)
            pd_error_total_output_net[o] = self.output_layer.neurons[o].calculate_total_net_error(train_outputs[o])
            
        pd_error_total_hidden_net = [0]*len(self.hidden_layer.neurons)
        for h in range(len(self.hidden_layer.neurons)):
            #calculate the ∂E(total_output_net)
            d_error_output_net = 0
            for o in range(len(self.output_layer.neurons)):
                d_error_output_net += pd_error_total_output_net[o]*self.output_layer.neurons[o].weights[h]
            #calculate the ∂E(total_output_net)/∂E(net_hidden)
            pd_error_total_hidden_net[h] = d_error_output_net*self.hidden_layer.neurons[h].calculate_pd_out_net_error()
            
        #renew the weights of hidden layer to output layer
        for o in range(len(pd_error_total_output_net)):
            for w in range(len(self.output_layer.neurons[o].weights)):
                pd_error_total_w_ho = pd_error_total_output_net[o]*self.output_layer.neurons[o].calculate_pd_net_input(w)
                
                self.output_layer.neurons[o].weights[w] -= self.Learning_Rate*pd_error_total_w_ho
                
        #renew the weights of input to hidden layer
        for h in range(len(pd_error_total_hidden_net)):
            for w in range(len(self.hidden_layer.neurons[h].weights)):
                pd_error_total_w_ih = pd_error_total_hidden_net[h]*self.hidden_layer.neurons[h].calculate_pd_net_input(w)
                
                self.output_layer.neurons[h].weights[w] -= self.Learning_Rate*pd_error_total_w_ih
                
    #calculate the total error
    def calculate_total_error(self,train_sets):
        total_error = 0
        for t in range(len(train_sets)):
            train_inputs, train_outputs = train_sets[t]
            self.feed_forward(train_inputs)
            for o in range(len(train_outputs)):
                total_error += self.output_layer.neurons[o].calculate_total_error(train_outputs[o])
        return total_error
    
        


In [220]:
class NeuronLayer:
    
    def __init__(self,num_neurons,bias):
        self.bias = bias
        self.neurons = []
        
        for n in range(num_neurons):
            self.neurons.append(Neurons(self.bias))
    
    def feed_forward(self,inputs):
        outputs = []
        for neuron in self.neurons:
            outputs.append(neuron.calculate_output(inputs))
        return outputs
    
    def get_outputs(self):
        outputs = []
        for neuron in self.neurons:
            outputs.append(neuron.output)
        return outputs


In [221]:
class Neurons:
    
    def __init__(self,bias):
        self.bias = bias
        self.weights = []
    
    def sigmoid(self, x):
        return 1/(1+math.exp(-x))
    
    def calculate_input_net(self):
        total = 0
        for i in range(len(self.inputs)):
            total += self.inputs[i]*self.weights[i]
        return total+self.bias
    
    def calculate_output(self,inputs):
        self.inputs = inputs
        self.output = self.sigmoid(self.calculate_input_net())
        return self.output    
    
    def calculate_total_error(self,target_output):
        return 0.5*(target_output - self.output)**2
        
    def calculate_pd_total_out_error(self,target_output):
        return (self.output - target_output)
    
    def calculate_pd_out_net_error(self):
        return self.output*(1-self.output)
        
    def calculate_total_net_error(self,target_output):
        return self.calculate_pd_total_out_error(target_output)*self.calculate_pd_out_net_error()
    
    def calculate_pd_net_input(self,i):
        return self.inputs[i]
     

In [222]:
nn = NeuralNetwork(2, 2, 2, hidden_layer_weights=[0.15, 0.2, 0.25, 0.3], hidden_layer_bias=0.35, output_layer_weights=[0.4, 0.45, 0.5, 0.55], output_layer_bias=0.6)

In [1]:
for i in range(10000):
    nn.train([0.05,0.1],[0.01,0.99])
    print(i,round(nn.calculate_total_error([[[0.05,0.1],[0.01,0.99]]]),6))
    print(nn.output_layer.get_outputs())

NameError: name 'nn' is not defined